# csv파일을 수정할때 주의 해야할 것들
## - 파일 위치 수정
## - 폴더명 수정
## 

In [2]:
import numpy as np
import pandas as pd
import glob
import os, sys
import matplotlib.pyplot as plt

os.chdir(r'C:\Users\pjk98\Data\New_Folder') # 파일 위치 입력
file_list = glob.glob("./*.csv", recursive=True)

folder_name = 'Sound_Test' # 새로운 폴더명
new_folder_path = os.path.join(os.getcwd(), folder_name) # 현재 폴더에서 새로운 폴더를 생성할 경로 생성
output = os.mkdir(new_folder_path) # CSV 파일 저장 디렉토리 경로 지정 후 폴더 생성

# names=['index','시각','음향2','Amb','Targ','del1','del2'], index_col='index'
# names=['Unnamed: 0','0','1','2','3','4','5']

i = 0
modify = 0
for file_name in file_list:
    df = pd.read_csv(file_list[i], sep=',',header=0,encoding='CP949')
    if len(df.columns) > 5:
        # 데이터프레임에서 특정 컬럼의 값이 존재하는 행만 지우기
        df = df.drop(df[df['4'].notna()].index)
        
        # 데이터프레임에서 결측치가 존재하는 행만 지우기
        nan_columns = ['1','2','3'] # '음향2','Amb','Targ'
        df.dropna(subset=nan_columns, inplace=True)
        
        # 데이터프레임의 특정 컬럼 유형 변환
        df['1'] = df['1'].astype(str)
        df['2'] = df['2'].astype(str)
        df['3'] = df['3'].astype(str)
        
        # 데이터프레임에서 특정 값이 들어간 행 제거
        df = df[~(df['1'].str.contains('A'))]
        df = df[~(df['2'].str.contains('A'))]
        df = df[~(df['3'].str.contains('A'))]
        df = df[~df['1'].str.endswith('.')]
        df = df[~df['2'].str.endswith('.')]
        df = df[~df['3'].str.endswith('.')]
        df = df[~df['1'].str.contains('\.\d+\.', regex=True)]
        df = df[~df['2'].str.contains('\.\d+\.', regex=True)]
        df = df[~df['3'].str.contains('\.\d+\.', regex=True)]
        
        # 숫자로 변환 후 내림하여 정수로 변환
        df['1'] = df['1'].astype(float).apply(np.floor).astype(int)
        
        # 데이터프레임의 특정 컬럼 유형 변환
        df['1'] = df['1'].astype(int)
        df['2'] = df['2'].astype(float)
        df['3'] = df['3'].astype(float)
        
        # 데이터프레임에서 특정 컬럼의 날짜 문자열이 유효하지 않은 행 제거
        df = df[pd.to_datetime(df['0'], errors='coerce').notna()]
        
        # 리스트에서 원하는 인덱스 위치에 있는 값들 제거하기
        col_list = df.columns.tolist()  # 컬럼 리스트를 리스트로 변환
        index_to_remove = [1, 2, 3, 4]  # 제거해서는 안되는 인덱스 위치 리스트
        for index in sorted(index_to_remove, reverse=True):
            col_list.pop(index) # 제거해서는 안되는 인덱스 위치를 리스트에서 제거합니다.
        drop_columns = col_list
        df.drop(labels=drop_columns, axis=1, inplace=True) # 불필요한 컬럼 제거
        
        modify += 1
    else:
        # 데이터프레임에서 특정 컬럼의 날짜 문자열이 유효하지 않은 행 제거
        df = df[pd.to_datetime(df['0'], errors='coerce').notna()]
        # 불필요한 컬럼 제거
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    df.columns=['Time','External Sound','Ambient Temp.','Target Temp.']
    df['Time'] = pd.to_datetime(df['Time']) # 시각 컬럼을 날짜/시간 형식으로 변환
    df['Time'] = df['Time'].dt.floor('S') # 1초 단위로 시간을 분할

    # timestamp = df['Time'].min().strftime('%Y%m%d')
    # folder_name = 'CMPS_'+timestamp # 새로운 폴더명
    # new_folder_path = os.path.join(os.getcwd(), folder_name) # 현재 폴더에서 새로운 폴더를 생성할 경로 생성
    # output = os.mkdir(new_folder_path) # CSV 파일 저장 디렉토리 경로 지정 후 폴더 생성

    # # 1초 단위로 나누어진 데이터프레임을 CSV 파일로 저장합니다.
    for _, group in df.groupby(pd.Grouper(key='Time', freq='1S')):
        columns_name = group.columns
        timestamp = group['Time'].min().strftime('%Y%m%d_%H%M%S')
        file = f'{new_folder_path}\{timestamp}.csv' # 파일 저장 위치 + 파일 명
        group.to_csv(file, header=columns_name, encoding='CP949')
    # !!!!!!!!!!!!!!!!!! os.remove(file_name) # 원래 CSV 파일을 삭제합니다. 이 코드는 사용금지!!!!!!!!!
    i += 1
print('문제가 있는 파일의 개수 :',modify+1)

문제가 있는 파일의 개수 : 2


In [105]:
# 리스트에서 원하는 인덱스 위치에 있는 값들 제거하기
col_list = df.columns.tolist()  # 컬럼 리스트를 리스트로 변환
index_to_remove = [1, 2, 3, 4]  # 제거할 인덱스 위치 리스트
print(col_list)

for index in sorted(index_to_remove, reverse=True):
    col_list.pop(index)

print(col_list)

['Unnamed: 0', '0', '1', '2', '3', '4', '5']
['Unnamed: 0', '4', '5']


In [134]:
timestamp = '987654'
folder_name = 'CMPS_'+timestamp # 새로운 폴더명
new_folder_path = os.path.join(os.getcwd(), folder_name) # 현재 폴더에서 새로운 폴더를 생성할 경로 생성
print(new_folder_path)
output = os.mkdir(new_folder_path) # CSV 파일 저장 디렉토리 경로 지정 후 폴더 생성

file = f'{new_folder_path}\{timestamp}.csv' # 파일 저장 위치 + 파일 명
print(file)

C:\Users\pjk98\Data\CMPS_23-05-31\CMPS_987654
C:\Users\pjk98\Data\CMPS_23-05-31\CMPS_987654\987654.csv
